In [ ]:
from selenium import webdriver
from time import sleep
import cdconfig
from bs4 import BeautifulSoup
import pandas as pd

driver=webdriver.Chrome(executable_path="C:\\Users\\elenc\\ChromeDriver\\chromedriver.exe")

# head over to garage and moving sales
driver.get("https://phoenix.craigslist.org/search/gms")
#driver.get("https://accounts.craigslist.org/login")
#sleep(3)

# switch to gridview to make grabbing many hrefs easier
try_click = driver.find_element_by_id("gridview").click()
sleep(1)
try_click = driver.find_element_by_id("picview").click()
sleep(2)
#login_form = driver.find_element_by_name("inputEmailHandle").send_keys(cdconfig.UNAME)

# gather up distinct listing urls to crawl
listing_urls = []
bs = BeautifulSoup(driver.page_source, 'html.parser')
block = bs.find(id="search-results")
for listing in block.find_all('a'):
    lurl= listing['href']
    if (lurl != '#') & (lurl not in listing_urls):
        listing_urls.append(lurl)
    
#login_form = driver.find_element_by_name("inputPassword").send_keys(cdconfig.PWORD)

#psource = driver.page_source
#now we could throw it into BeautifulSoup!
#print(psource)
